In [1]:
import pandas as pd
import ydata_profiling as yp
import re

df = pd.read_excel('original/productos_departamento.xlsx')
# df.info()

c:\Users\admin\Documents\acoding\HanovaCaso2\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dfi = df.astype(str)  # convierte el reporte inicial todo a texto para poder mostrarlo sin errores
dfi.to_excel('sample/productos_departamento_object.xlsx', index=False, engine='openpyxl')

In [3]:
# Product_Name 
df['Product_Name'] = df['Product_Name'].str.title().str.replace('_', ' ', regex=False)

In [4]:
# Price
def normalize_price(value):
    if pd.isna(value):
        return None
    
    # Extrae numero con o sin comas y punto decimal
    match = re.search(r'([\d,]+(?:\.\d+)?)', str(value))
    if match:
        num = match.group(1).replace(',', '') # Quitamos comas 
        try:
            return float(num)
        except ValueError:
            return None
    return None

df['Price'] = df['Price'].apply(normalize_price)

In [5]:
# Category, Subcategory

# Buscamos valores que contengan un guion - en Category para separarlos
df_mask = df['Category'].str.contains('-', na=False)

df.loc[df_mask, 'Subcategory'] = df.loc[df_mask, 'Category'].str.split('-', n=1).str[1] # Izq -> Category
df.loc[df_mask, 'Category'] = df.loc[df_mask, 'Category'].str.split('-', n=1).str[0] # Der -> Subcategory

# Capitalizamos y normalizamos nombres
df['Subcategory'] = df['Subcategory'].str.title().replace({'Pantalón': 'Pantalones', 'Nan': None})
df['Category'] = df['Category'].str.title().replace('Vestimenta', 'Ropa')


In [6]:
# Stock
df['Stock'] = pd.to_numeric(df['Stock'], errors='coerce')

In [7]:
# Solución con dateutil.parser (no utilizada)

# from dateutil import parser
# df['Last_Update'] = df['Last_Update'].apply(
#     lambda x: parser.parse(x, dayfirst=True) if pd.notna(x) and str(x).lower() != 'nan' else pd.NaT
# )

# Primer intento: dayfirst=True DMY
dt = pd.to_datetime(df['Last_Update'], errors='coerce', dayfirst=True)

# Segundo intento YMD
dt_mask = dt.isna() & df['Last_Update'].notna() # Los que fallaron en la primera y que no eran NaT
dt2 = pd.to_datetime(df.loc[dt_mask, 'Last_Update'], errors='coerce', dayfirst=False)

# Combina resultados
dt.loc[dt_mask] = dt2
df['Last_Update'] = dt


In [8]:
# Creamos ProductID como primer columna
df.insert(0, 'Product_ID', range(1001, 1001 + len(df)))
df['Product_ID'] = df['Product_ID'].astype('int64')

In [9]:
# ERP_Code
# Cargar categorias_maestras
cat_df = pd.read_excel('limpio/categorias_maestras.xlsx', dtype=str)

# Left merge para obtener ERP_Code donde Category y Subcategory coinciden
df = df.merge(
    cat_df[['Category_Clean', 'Subcategory_Clean', 'ERP_Code']],
    left_on=['Category', 'Subcategory'],
    right_on=['Category_Clean', 'Subcategory_Clean'],
    how='left'
)

# Insertar ERP_Code después de Category y Subcategory
df['ERP_Code'] = df['ERP_Code'].astype('object')
erp_col = df.pop('ERP_Code')
df.insert(6, 'ERP_Code', erp_col)

# Dropeamos columnas auxiliares
df = df.drop(columns=['Category_Clean', 'Subcategory_Clean'])

In [10]:
# Generar reporte pandas profiling
report = yp.ProfileReport(
            df,
            title="Reporte Productos Departamento",
            explorative=True,
            minimal=True,
        )
report.to_file("limpio/reporte_productos_departamento.html")

Export report to file: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [11]:
# num_duplicates = df.duplicated().sum()
# print(f"Num filas duplicadas: {num_duplicates}")
# df = df.drop_duplicates()

df.to_excel('limpio/productos_departamento_clean.xlsx', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Product_ID    500 non-null    int64         
 1   Product_Name  500 non-null    object        
 2   SKU           480 non-null    object        
 3   Price         481 non-null    float64       
 4   Category      500 non-null    object        
 5   Subcategory   383 non-null    object        
 6   ERP_Code      378 non-null    object        
 7   Stock         452 non-null    float64       
 8   Provider_ID   500 non-null    int64         
 9   Last_Update   486 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 39.2+ KB
